## Let's train some simple models in pure nnvm

In [1]:
import nnvm
import nnvm.symbol as sym
from nnvm.compiler.graph_util import gradients
from nnvm.compiler.optimizer import SGD

import tvm
from tvm.contrib import graph_runtime

import numpy as np
from matplotlib import pyplot as plt

We need keras because it is the easiest way to load datasets I know.

In [2]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

/nix/store/p3i9s3vhjskbrnfl97fd7b0vmn7bqddh-python3.6-h5py-2.7.1/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def batches(batch_size, x=x_train, y=y_train, repeat=True):
    while True:
        for i in range(int(x.shape[0] / batch_size)):
            yield (x[i:i+batch_size, ...].astype('float32')/255.0, 
                   np.eye(10)[y[i:i+batch_size]].astype('float32'))
        if not repeat:
            return

Define a neural net

In [4]:
BATCH_SIZE = 32

v_images = sym.Variable("images", shape=(BATCH_SIZE, 1, 28, 28), dtype=0)
v_true_labels = sym.Variable("true_labels", shape=(BATCH_SIZE, 10), dtype=0)

x = v_images
x = sym.reshape(data=x, shape=(BATCH_SIZE, 28*28))
x = sym.dense(data=x, units=10)
logits = x

x = - sym.elemwise_mul(v_true_labels, sym.log_softmax(x))
loss = sym.sum(x) / BATCH_SIZE

# This is not really accuracy, because we use softmax instead of hardmax
accuracy = sym.sum(v_true_labels * sym.softmax(logits)) / BATCH_SIZE

Create two graphs: `graph` for training and `forward_graph` for inference

In [5]:
# We have to somehow list all weights (the corresponding variables are generated automatically)
weight_vars = [v for v in loss.list_input_variables() if v.attr('name') not in ['images', 'true_labels']]

optimizer = SGD(learning_rate=1e-4)
update_step = optimizer.minimize(loss, var=weight_vars)

graph = nnvm.graph.create(sym.Group([loss, update_step])).apply("InferShape").apply("InferType")
forward_graph = nnvm.graph.create(sym.Group([loss, accuracy])).apply("InferShape").apply("InferType")

In [6]:
#print(graph.ir(join_node_attrs=['shape', 'dtype']))

Compile both graphs.

In [7]:
cgraph, libmod, params = nnvm.compiler.build(graph, 'llvm')
m = graph_runtime.create(cgraph, libmod, tvm.cpu(0))
fcgraph, flibmod, fparams = nnvm.compiler.build(forward_graph, 'llvm')
fm = graph_runtime.create(fcgraph, flibmod, tvm.cpu(0))

Randomly initialize weights.

In [8]:
if params:
    m.set_input(**params)
    
if fparams:
    fm.set_input(**fparams)

shapes = graph.json_attr('shape')
    
for v in loss.list_input_variables():
    shape = shapes[graph.index.node_id(v.attr('name'))]
    print("Initializing " + str(v.attr('name')) + " " + str(shape))
    m.set_input(v.attr('name'), np.random.normal(scale=0.1, size=shape).astype('float32'))

Initializing true_labels [32, 10]
Initializing images [32, 1, 28, 28]
Initializing dense0_weight [10, 784]
Initializing dense0_bias [10]


This functions runs the forward graph on the test data using current weights from the training graph.

In [9]:
def test():
    # copy weights from training to inference. Not sure if we can avoid it and still run only the inference part (without grads)
    for v in weight_vars:
        shape = shapes[graph.index.node_id(v.attr('name'))]
        # note that we use get_input: _assign mutates the input variable
        fm.set_input(v.attr('name'), m.get_input(v.attr('name'), tvm.nd.empty(shape)))
    
    loss = []
    acc = []
    
    for b in batches(BATCH_SIZE, x_test, y_test, repeat=False):
        fm.set_input('images', b[0][:, None, ...])
        fm.set_input('true_labels', b[1])
        fm.run()
        loss.append(fm.get_output(0, tvm.nd.empty((1,))).asnumpy()[0])
        acc.append(fm.get_output(1, tvm.nd.empty((1,))).asnumpy()[0])
    
    return np.mean(loss), np.mean(acc)

In [10]:
seen = 0
tr_loss = np.inf
for i, b in enumerate(batches(BATCH_SIZE)):
    if i % 1000 == 0:
        print("step:", i, "seen:", seen, "tr loss:", tr_loss)
        
        if i % 10000 == 0:
            l, a = test()
            print("test loss:", l, "test accuracy:", a)
    
    # load data
    m.set_input('images', b[0][:, None, ...])
    m.set_input('true_labels', b[1])
    # run a training step
    m.run()
    
    seen += b[0].shape[0]
    tr_loss = m.get_output(0, tvm.nd.empty((1,))).asnumpy()

step: 0 seen: 0 tr loss: inf
test loss: 2.605138 test accuracy: 0.104033045
step: 1000 seen: 32000 tr loss: [2.5119948]
step: 2000 seen: 64000 tr loss: [2.2600343]
step: 3000 seen: 96000 tr loss: [2.3432593]
step: 4000 seen: 128000 tr loss: [2.0252519]
step: 5000 seen: 160000 tr loss: [1.9078416]
step: 6000 seen: 192000 tr loss: [1.8912483]
step: 7000 seen: 224000 tr loss: [1.8423703]
step: 8000 seen: 256000 tr loss: [1.8401768]
step: 9000 seen: 288000 tr loss: [1.888246]
step: 10000 seen: 320000 tr loss: [1.7447925]
test loss: 1.8121912 test accuracy: 0.19732629
step: 11000 seen: 352000 tr loss: [1.4131862]
step: 12000 seen: 384000 tr loss: [1.6707991]
step: 13000 seen: 416000 tr loss: [1.4810647]
step: 14000 seen: 448000 tr loss: [1.3933669]
step: 15000 seen: 480000 tr loss: [1.4294233]
step: 16000 seen: 512000 tr loss: [1.4877458]
step: 17000 seen: 544000 tr loss: [1.4209709]
step: 18000 seen: 576000 tr loss: [1.6107997]
step: 19000 seen: 608000 tr loss: [1.2538222]
step: 20000 seen

KeyboardInterrupt: 